In [1]:
import AFIPy as AFIP

In [2]:
# Paquetes.
import shutil
import os
import time
import pandas as pd
from typing import Any
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, Alignment
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# Fechas a facturar.
Fechas = AFIP.Generate_Previous_Days_List()

['26/12/2024', '27/12/2024', '28/12/2024', '29/12/2024']

In [ ]:
# Variables globales.
Ruta_Descarga_Payway = 'C:/Users/tomas/Downloads'
Nombre_Viejo = 'Movimientos En Linea Delimitado por comas.csv'
Ruta_Nueva_Payway = 'J:/My Drive/Forraje/AFIP' 
Nombre_Nuevo = 'Payway.csv'
Email_Payway = 'carolina8101924@gmail.com'
Contraseña_Payway = '123Nogue$'
Usuario_AFIP = '27202147025'
Contraseña_AFIP = '123Patricio$'
Empresa = 'MARQUEZ CAROLINA MARIEL'
Punto_de_Ventas = '00002-Las Piedras 2837 - Kilometro 45, Buenos Aires'
Tipo_Comprobante = 'Factura C'
Tipo_Concepto = 'Productos'
Tipo_Condicion_IVA = 'Consumidor Final'
Tipo_Pago = 'Contado'
Tipo_Comprobante_Valor = AFIP.Tipos_de_Comprobantes.get(Tipo_Comprobante)
Tipo_Concepto_Valor = AFIP.Tipos_de_Conceptos.get(Tipo_Concepto)
Tipo_Condicion_IVA_Valor = AFIP.Tipos_Condicion_IVA.get(Tipo_Condicion_IVA)

In [5]:
# Abrir pestaña de Payway.
Payway = AFIP.Initialize_Chrome_Driver()

In [6]:
# Descargar documento de Payway.
AFIP.Download_CSV_From_Payway(Payway, Email_Payway, Contraseña_Payway)

<selenium.webdriver.chrome.webdriver.WebDriver (session="c846b3fcb6c85c0b9219894c19e6f623")>

In [7]:
# Esperar descarga del CSV.
AFIP.Wait_For_Download(f'{Ruta_Descarga_Payway}/{Nombre_Viejo}')

In [8]:
# Mover y renombrar archivo CSV.
AFIP.Move_And_Rename_File(Ruta_Descarga_Payway, Nombre_Viejo, Ruta_Nueva_Payway, Nombre_Nuevo)

In [5]:
# Crear DataFrame del CSV.
df = pd.read_csv(Ruta_Nueva_Payway + '/' + Nombre_Nuevo, skiprows=1)

In [6]:
# Dividir filas con valores mayores a 100000.
df = AFIP.Split_Rows_By_Threshold(df, 'MONTO_BRUTO', 100000)

In [7]:
# Retocar CSV para que quede preparado para su utilización.
AFIP.Process_And_Save_Dataframe(df, f'{Ruta_Nueva_Payway}/AFIP.xlsx')

In [8]:
# Ruta del archivo con los precios a subir.
Archivo = f'{Ruta_Nueva_Payway}/AFIP.xlsx'

In [9]:
# Crear DataFrame final para facturar.
df = pd.read_excel(Archivo)

In [10]:
# Filtrar df con las fechas especificadas al comienzo.
df = df[df['Fecha'].isin(Fechas)]

In [11]:
# Resetear el índice del DataFrame.
df = df.reset_index(drop=True)

In [12]:
# Variables para las columnas.
Fecha = df['Fecha']
Descripcion = df['Descripción']
Precio = df['Precio']

In [13]:
# Abrir pestaña de AFIP.
Afip = AFIP.Initialize_Chrome_Driver()

In [14]:
# Loguearse en AFIP.
AFIP.Login_To_AFIP(Afip, Usuario_AFIP, Contraseña_AFIP, Empresa)

<selenium.webdriver.chrome.webdriver.WebDriver (session="56b9e798f559a7e3c64243176da0d77a")>

In [15]:
# Bucle de facturación en AFIP producto a producto.
for i in range (0, len(Descripcion)):
    AFIP.Generate_Invoice(Afip, 
                          Punto_de_Ventas, 
                          Tipo_Comprobante_Valor,
                          Fecha[i], 
                          Tipo_Concepto_Valor, 
                          Tipo_Condicion_IVA_Valor, 
                          Tipo_Pago, 
                          Descripcion[i], 
                          Precio[i])

In [ ]:
# Cerrar todas las pestañas.
AFIP.Close_All_Chrome_Tabs(Afip)
AFIP.Close_All_Chrome_Tabs(Payway)